# 🧠 Task 1: Text Agent (Research Analyst)

### Objective
Before building the complete agent, I am first creating a **baseline LLM model** that can **summarize paragraphs** from research papers.  
This step helps establish the core functionality (text understanding and summarization) that will later be integrated into the **Text Agent** component of the CellSense system.

### Plan
1. Implement a simple LLM pipeline with:
   - Model configuration  
   - Prompt template  
   - Output parser (for structured summaries)
2. Test the model by summarizing random paragraphs from sample papers.
3. Analyze the outputs and refine the prompt for clarity and accuracy.
4. Once validated, extend the pipeline to include:
   - Document Loaders  
   - Text Splitters  
   - Memory and Tools  
   - Agent orchestration (in later stages)

### Outcome
A working summarization prototype that serves as the **foundation for the Text Agent**, capable of analyzing scientific text before integrating with other agents in CellSense.


#### Model Configuration

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"


In [2]:
from langchain.chat_models import init_chat_model
from langchain.messages import SystemMessage,HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from dotenv import load_dotenv
import os

# Loading the environment variables, such as openAI API key. This will keep my API key secret.

load_dotenv()

model = init_chat_model("openai:gpt-4o-mini", 
                        temperature = 0.8, 
                        max_tokens = 2000, 
                        max_retries = 2,
                        timeout = 60)

/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/CellSense/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Prompt Template

In [3]:
system_message = '''
You are an expert biomedical research assistant specializing in analyzing scientific literature. 
Your task is to carefully read the provided paragraph and perform the following tasks:

1. Summarize the paragraph in detail and meaningful way.
2. Extract the following information if mentioned:
   - Key concepts or keywords
   - Material(s) being studied
   - Methods or experimental techniques
   - Parameters to consider
   - Technology or tools used

3. If a field is not mentioned in the paragraph, return an empty list for that field.

Guidelines:
- Be precise and concise in the summary.
- Extract only what is explicitly mentioned in the paragraph.
- Avoid adding assumptions or information not present in the text.
'''

In [4]:
template = ChatPromptTemplate([
    ("system", system_message),
    ("user", "Can you please summarise the given text?: {text}")
])

Paragraph = """An extensive research field in regenerative medicine is electrical stimulation (ES) and its
impact on tissue and cells. The mechanism of action of ES, particularly the role of electrical parameters
like intensity, frequency, and duration of the electric field, is not yet fully understood. Human MG-63
osteoblasts were electrically stimulated for 10 min with a commercially available multi-channel
system (IonOptix). We generated alternating current (AC) electrical fields with a voltage of 1 or 5 V
and frequencies of 7.9 or 20 Hz, respectively. To exclude liquid-mediated effects, we characterized the
AC-stimulated culture medium. AC stimulation did not change the medium's pH, temperature, and
oxygen content. The H2O2 level was comparable with the unstimulated samples except at 5 V_7.9 Hz,
where a significant increase in H2O2 was found within the first 30 min. Pulsed electrical stimulation
was beneficial for the process of attachment and initial adhesion of suspended osteoblasts. At the
same time, the intracellular Ca2+ level was enhanced and highest for 20 Hz stimulated cells with
1 and 5 V, respectively. In addition, increased Ca2+ mobilization after an additional trigger (ATP)
was detected at these parameters. New knowledge was provided on why electrical stimulation
contributes to cell activation in bone tissue regeneration"""
prompt = template.invoke({"text" : Paragraph})

print(prompt)

messages=[SystemMessage(content='\nYou are an expert biomedical research assistant specializing in analyzing scientific literature. \nYour task is to carefully read the provided paragraph and perform the following tasks:\n\n1. Summarize the paragraph in detail and meaningful way.\n2. Extract the following information if mentioned:\n   - Key concepts or keywords\n   - Material(s) being studied\n   - Methods or experimental techniques\n   - Parameters to consider\n   - Technology or tools used\n\n3. If a field is not mentioned in the paragraph, return an empty list for that field.\n\nGuidelines:\n- Be precise and concise in the summary.\n- Extract only what is explicitly mentioned in the paragraph.\n- Avoid adding assumptions or information not present in the text.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content="Can you please summarise the given text?: An extensive research field in regenerative medicine is electrical stimulation (ES) and its\nimpact on tissue and

#### Testing model Output

In [5]:
response = model.invoke(prompt)
print(response.content)

KeyboardInterrupt: 

#### 👩‍🏫 key points:

* I have sent only the abstract from the paper and it summarised well.

* The summary and other details are very helpful, but this is in plain text format, if I want to send this details or another details to another model or an agent in the future, it won't understand it should follow a structured way may be `JSON` or similar like strucure.

#### Trying to get structured output

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional,Literal,List

# Define structured output schema
class ExtractUsefulInfo(BaseModel):
    summary: str = Field(description="Detailed summary of the text")
    Keywords: Optional[List[str]] = Field(default=[], description="Important concepts")
    Materials: Optional[List[str]] = Field(default=[], description="Materials used")
    Methods: Optional[List[str]] = Field(default=[], description="Methods or experimental techniques")
    Parameters: Optional[List[str]] = Field(default=[], description="Experimental parameters")
    Technology: Optional[List[str]] = Field(default=[], description="Technologies or tools used")

formated_prompt = template.format(text=Paragraph)
# Creating a strucutred wrapper around the model
structured_model = model.with_structured_output(ExtractUsefulInfo)

response = structured_model.invoke(formated_prompt)


KeyboardInterrupt: 

In [ ]:
print(type(response))

<class '__main__.ExtractUsefulInfo'>


#### Writing all as a chain

In [ ]:
from langchain_core.runnables import RunnableSequence
chain = RunnableSequence(template, structured_model)
response_chain = chain.invoke({"text": Paragraph})
print(type(response_chain))
print(response_chain.Materials)
print(response_chain.summary)

<class '__main__.ExtractUsefulInfo'>
['human MG-63 osteoblasts', 'culture medium']
This research focuses on the effects of electrical stimulation (ES) on tissue and cells within the field of regenerative medicine, particularly its influence on human MG-63 osteoblasts. The study investigates how different electrical parameters such as intensity, frequency, and duration affect the mechanism of action of ES, which is not fully understood. Using a multi-channel system (IonOptix), the researchers applied alternating current (AC) electrical fields at voltages of 1 or 5 V and frequencies of 7.9 or 20 Hz for 10 minutes. They also characterized the AC-stimulated culture medium to rule out liquid-mediated effects, finding that stimulation did not alter pH, temperature, or oxygen content, except for a significant increase in H2O2 levels at 5 V and 7.9 Hz. The results indicated that pulsed electrical stimulation promoted the attachment and initial adhesion of osteoblasts, elevated intracellular Ca


Trying to understand whether the model can process each keywords and understand based on the previous context.

In [ ]:
from langchain_core.runnables import RunnableSequence, RunnablePassthrough, RunnableParallel

# Here, we need to define system message again because it doesn't remember from earlier prompt.
template_1 = ChatPromptTemplate([("system", system_message),
                               ("user", " Explain the {Keywords} in detail")]) 

chain = RunnableSequence(template, structured_model, template_1, )

#### 👩‍🏫 key points:

* Here is one problem now, As I want to pass the another prompt, I have to define ne system message for model as it doesn't remember the previous context.

* Now, I have to either 
    * provide new system message to another model defining it's role so it doesn't get confused with it's new task 
    * or provide a memory 
    * or I have to use tool calling but this will come in agent
    * or I have to chose another aproach, which is RAG (Retrieval-Augmented Generation)

* So, Staring with RAG, where I need to first upload paper, split texts, generate embeddings.


# RAG (Retrieval-Augmented Generation)

* RAG will help me to overcome the above problem which is limited context and static knowledge-doesn't automatically update when new events, aftcs occur.

* How it will help?

    - As I upload a document it will fetch a relevant information from that document and provide me result related to the query and the context. In simple terms, it allos to use external knowledge that it didn't learn while training.

* To Create a RAG-based agent,I first need to create knowledge base, which is a repository of all the documents or structured data.

### RAG based application is comprise of four main components:

    1. Document Loader
    2. Text splitter
    3. Vector Databases
    4. Retrievers


## Document Loader 
First thing to start with is, How do we load a document. A document can be pdf, word file, or any file.

#### 👩‍🏫 key points:
* Data can be coming from various sources, such as pdf, text, google drive, slack, amazon s3, and the chances are high that each source might have different format. LangChain providea common interface for reading data regardless of the sources
* There are hundreds of document loaders. But for Text Agent, I have research paper in pdf format. So I will use pdfloader.

* I am planning to use UnstructuredPDFLoader. As Research Papers are complex PDFs, containing two columnn layout, figures, tables, captions, footnotes, we need to use unstructuredPDFLoader, helping for multimodal RAG.

### Experimenting with Langchain's UnstructuredPDFLoader

In [ ]:

# Importing pdfloader
from langchain_community.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader("/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/Documents/cells-11-02650-v2.pdf", 
                               mode="elements",
                               strategy="hi_res",
                               infer_table_structure=True,
                               extract_images_in_pdf=True,                           
                               extract_image_block_types=["Image"],  # if want table as an image then we can add table also
                               extract_image_block_to_payload=True,
                               )

docs = loader.load() # It will load the document


/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/CellSense/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [ ]:
for doc in docs[20:40]:   print(doc)

NameError: name 'docs' is not defined

In [ ]:
for doc in docs[2:20]:
    print(f"category:{doc.metadata.get('category')}----> content:{doc.page_content}, Page_number: {doc.metadata.get('page_number')}")

NameError: name 'docs' is not defined

In [ ]:
# Cheking the categories in Document Objects
for doc in docs:
    list_categories = [doc.metadata.get('category') for doc in docs]

print(set(list_categories))

{'ListItem', 'UncategorizedText', 'FigureCaption', 'Header', 'NarrativeText', 'Table', 'Image', 'Title'}


#### 👩‍🏫 key points:

* Here, It is important to know what type of category the text is divided into. This will help to find the different elements of a pdf is classified in what categories.

#### Checking the metadata based on categories

In [ ]:
# Table Category
for doc in docs[40:60]:
    if doc.metadata.get('category') == 'Table':
        print(f"{doc.metadata.get("page_number")}")
        print(f"**Content** ===> {doc.page_content}")
        print(doc.metadata)
        print(f"metadata keys----> {list(doc.metadata.keys())}\n")
        



9
**Content** ===> Voltage [V] - 1 V 1 V 5 V 5 V Frequency [Hz] Pulse Duration [ms] Electric Field [V/m] * Theoretical Electric Current [A] * Measured Electric Current [A] ** Code - - - - - 20 3.6 90 0.0077 0.012 7.9 10 90 0.0077 0.012 20 3.6 450 0.0385 0.067 7.9 10 450 0.0385 0.069
{'source': '/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/Documents/cells-11-02650-v2.pdf', 'detection_class_prob': 0.91845703125, 'is_extracted': 'true', 'coordinates': {'points': ((np.float64(120.21004486083984), np.float64(359.53082275390625)), (np.float64(120.21004486083984), np.float64(653.4051513671875)), (np.float64(1539.0557861328125), np.float64(653.4051513671875)), (np.float64(1539.0557861328125), np.float64(359.53082275390625))), 'system': 'PixelSpace', 'layout_width': 1654, 'layout_height': 2339}, 'last_modified': '2025-11-01T08:41:48', 'text_as_html': '<table><thead><tr><th></th><th>Voltage IV]</th><th>Frequency (Hz]</th><th>PulseDuration [ms]</th><th>Electric Field [Vim] *</th><th>Theore

#### 👩‍🏫 key points:

* For Table category, the DocumnetLoader has clearly identified tables and it has contained all the elements are present in the table. We can use this content to create a chunk and generate embedding to store in vector database.

* It is important to check the metadata also, which is also gonna be stored in vector database along with embeddingsa this will help us to provide additional information at query time. 

    > For example, *"The table in cells-11-02650-v2.pdf(source), on page number 14 illustrates that........."*

* The metadata key is to get the overview what type of metadata is available so, we can then decide which keys to include in metadata and which to remove.

In [ ]:
# Title Category
for doc in docs[60:80]:
    if doc.metadata.get('category') == 'Title':
        print(f"{doc.metadata.get("page_number")}")
        print(f"**Content** ===> {doc.page_content}")
        print(doc.metadata)
        print(f"metadata keys----> {list(doc.metadata.keys())}\n")

1
**Content** ===> Pulsed Electrical Stimulation Affects Osteoblast Adhesion and Calcium Ion Signaling
{'source': '/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/Documents/cells-11-02650-v2.pdf', 'detection_class_prob': 0.6241938471794128, 'is_extracted': 'true', 'coordinates': {'points': ((np.float64(96.58751678466797), np.float64(322.30853271484375)), (np.float64(96.58751678466797), np.float64(432.4170133333328)), (np.float64(1523.1252444444451), np.float64(432.4170133333328)), (np.float64(1523.1252444444451), np.float64(322.30853271484375))), 'system': 'PixelSpace', 'layout_width': 1654, 'layout_height': 2339}, 'last_modified': '2025-11-01T08:41:48', 'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 1, 'file_directory': '/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/Documents', 'filename': 'cells-11-02650-v2.pdf', 'category': 'Title', 'element_id': '2872ad1a86c439ba151f452a6f97adef'}
metadata keys----> ['source', 'detection_class_prob', 'is_extracted', '

#### 👩‍🏫 key points:

* We need to create embeddings for Title category as it provides valuable context-The information about the following paragraph. As we can see, the document loader has correctly classified almost all the title but there are some noise too.

* Again for this category we need to include only few metadata, such as source, coordinates, page_number.

In [ ]:
# Header Category
for doc in docs[80:100]:
    if doc.metadata.get('category') == 'Header':
        print(f"{doc.metadata.get("page_number")}")
        print(f"**Content** ===> {doc.page_content}")
        print(doc.metadata)
        print(f"metadata keys----> {list(doc.metadata.keys())}\n")

2
**Content** ===> Cells 2022, 11, 2650
{'source': '/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/Documents/cells-11-02650-v2.pdf', 'detection_class_prob': 0.7379001975059509, 'is_extracted': 'true', 'coordinates': {'points': ((np.float64(97.15570068359375), np.float64(158.2140350341797)), (np.float64(97.15570068359375), np.float64(181.1849116666669)), (np.float64(282.9130859375), np.float64(181.1849116666669)), (np.float64(282.9130859375), np.float64(158.2140350341797))), 'system': 'PixelSpace', 'layout_width': 1654, 'layout_height': 2339}, 'last_modified': '2025-11-01T08:41:48', 'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 2, 'file_directory': '/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/Documents', 'filename': 'cells-11-02650-v2.pdf', 'category': 'Header', 'element_id': '5c72fa6a292e875d7cf7227febce387c'}
metadata keys----> ['source', 'detection_class_prob', 'is_extracted', 'coordinates', 'last_modified', 'filetype', 'languages', 'page_number', '

#### 👩‍🏫 key points:

* I thinnk, we don't need this category to be stored in vectorstore or vector database, while it only contains page numbers and page headers.

In [ ]:
# NarrativeText Category
for doc in docs[30:50]:
    if doc.metadata.get('category') == 'NarrativeText':
        print(f"{doc.metadata.get("page_number")}")
        print(f"**Content** ===> {doc.page_content}")
        print(doc.metadata)
        print(f"metadata keys----> {list(doc.metadata.keys())}\n")


1
**Content** ===> Article
{'source': '/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/Documents/cells-11-02650-v2.pdf', 'detection_class_prob': 0.8227533102035522, 'is_extracted': 'true', 'coordinates': {'points': ((np.float64(97.11119842529297), np.float64(290.358642578125)), (np.float64(97.11119842529297), np.float64(321.78399611111104)), (np.float64(178.90414428710938), np.float64(321.78399611111104)), (np.float64(178.90414428710938), np.float64(290.358642578125))), 'system': 'PixelSpace', 'layout_width': 1654, 'layout_height': 2339}, 'last_modified': '2025-11-01T08:41:48', 'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 1, 'file_directory': '/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/Documents', 'filename': 'cells-11-02650-v2.pdf', 'category': 'NarrativeText', 'element_id': 'be3a187cfe3ab16d142f7c7dfc25ceeb'}
metadata keys----> ['source', 'detection_class_prob', 'is_extracted', 'coordinates', 'last_modified', 'filetype', 'languages', 'page_number',

#### 👩‍🏫 key points:

* This category is very crucial as it is the main source of information. As we can see all the paragraphs from the pdf is classified into this category.

* We can include only similir metadata as mentioned above.

In [ ]:
# ListItem Category

for doc in docs[10:20]:
    if doc.metadata.get('category') == 'ListItem':
        print(f"{doc.metadata.get("page_number")}")
        print(f"**Content** ===> {doc.page_content}")
        print(doc.metadata)
        print(f"metadata keys----> {list(doc.metadata.keys())}\n")

1
**Content** ===> 1 Department of Cell Biology, Rostock University Medical Center, 18057 Rostock, Germany
{'source': '/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/Documents/cells-11-02650-v2.pdf', 'detection_class_prob': 0.838455855846405, 'is_extracted': 'true', 'coordinates': {'points': ((np.float64(458.4291687011719), np.float64(598.7752311111112)), (np.float64(458.4291687011719), np.float64(624.9765783333335)), (np.float64(1377.2996826171875), np.float64(624.9765783333335)), (np.float64(1377.2996826171875), np.float64(598.7752311111112))), 'system': 'PixelSpace', 'layout_width': 1654, 'layout_height': 2339}, 'last_modified': '2025-11-01T08:41:48', 'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 1, 'file_directory': '/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/Documents', 'filename': 'cells-11-02650-v2.pdf', 'parent_id': '8f6456b57c49f15b5e0bb5356a1bc108', 'category': 'ListItem', 'element_id': '6f2dc602c3af9558ae2cd8c09640c9a5'}
metadata keys---->

#### 👩‍🏫 key points:

* This category includes only references and the items that are present in the list. Hence, I've decided not to consider

In [ ]:
# UncategorizedText Category
for doc in docs[0:10]:
    if doc.metadata.get('category') == 'UncategorizedText':
        print(f"{doc.metadata.get("page_number")}")
        print(f"**Content** ===> {doc.page_content}")
        print(doc.metadata)
        print(f"metadata keys----> {list(doc.metadata.keys())}\n")
        


1
**Content** ===> ae wo
{'source': '/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/Documents/cells-11-02650-v2.pdf', 'coordinates': {'points': ((np.float64(451.0), np.float64(684.0)), (np.float64(451.0), np.float64(760.0)), (np.float64(479.0), np.float64(760.0)), (np.float64(479.0), np.float64(684.0))), 'system': 'PixelSpace', 'layout_width': 1654, 'layout_height': 2339}, 'last_modified': '2025-11-01T08:41:48', 'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 1, 'file_directory': '/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/Documents', 'filename': 'cells-11-02650-v2.pdf', 'parent_id': '8f6456b57c49f15b5e0bb5356a1bc108', 'category': 'UncategorizedText', 'element_id': '57b0174c00d8f350de0c4629208cf463'}
metadata keys----> ['source', 'coordinates', 'last_modified', 'filetype', 'languages', 'page_number', 'file_directory', 'filename', 'parent_id', 'category', 'element_id']

4
**Content** ===> ∆φ = 0
{'source': '/home/vraj/Desktop/CellSense/X20251016-CellSen

#### 👩‍🏫 key points:

* This category includes only reference numbers and the garbage. Hence, I've decided not to consider

In [ ]:
# Image Category
for doc in docs[40:70]:
    if doc.metadata.get('category') == 'Image':
        print(f"{doc.metadata.get("page_number")}")
        print(f"**Content** ===> {doc.page_content}")
        print(doc.metadata)
        print(f"metadata keys----> {list(doc.metadata.keys())}\n")

1
**Content** ===> f cells
{'source': '/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/Documents/cells-11-02650-v2.pdf', 'detection_class_prob': 0.5524067878723145, 'coordinates': {'points': ((np.float64(98.8648681640625), np.float64(136.2649383544922)), (np.float64(98.8648681640625), np.float64(230.7023162841797)), (np.float64(331.4766540527344), np.float64(230.7023162841797)), (np.float64(331.4766540527344), np.float64(136.2649383544922))), 'system': 'PixelSpace', 'layout_width': 1654, 'layout_height': 2339}, 'last_modified': '2025-11-01T08:41:48', 'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 1, 'image_base64': '/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCABfAOgDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fA

#### 👩‍🏫 key points:

* This category is for all the images present in the pdf. we need to consider it as it is another crucial element.

* The metadata should conain source, page number, coordinates, image_base64, image_mime_type.

* This metadata will help to get the image from the pdf to enhance the context. Though image itself not sent to llm for reasoning.


In [ ]:
# FigureCaption Category

for doc in docs[40:70]:
    if doc.metadata.get('category') == 'FigureCaption':
        print(f"{doc.metadata.get("page_number")}")
        print(f"**Content** ===> {doc.page_content}")
        print(doc.metadata)
        print(f"metadata keys----> {list(doc.metadata.keys())}\n")

7
**Content** ===> Figure 2. Scheme of experimental approaches of AC stimulation and calcium ions (Ca2+)-imaging in confocal microscopy. At ﬁrst, we stained the suspended MG-63s with ﬂuo3/AM for 40 min. Stained cells were then seeded into the 12-well plate under the microscope (LSM780, Carl Zeiss). Next, the electrodes were placed, and the electrical stimulation started. After a short settling phase of cells, a time series was done to measure the basal Ca2+ activity during ES (60 s). After 10 min ES, the electrodes were removed, and a second time series was recorded immediately to analyze the Ca2+ release and reaction after ATP addition (480 s; 240 cycles each 2 s, after 180 s ATP addition). (Parts of the ﬁgure were drawn by using pictures from Servier Medical Art. Servier Medical Art by Servier is licensed under a Creative Commons Attribution 3.0 Unported License (https://creativecommons. org/licenses/by/3.0/, accessed on 21 July 2022)).
{'source': '/home/vraj/Desktop/CellSense/X20251

#### 👩‍🏫 key points:

* This will help to connecting to the image info or table information which will provide the context with the finding.

### Notes:

* To create a chunks UncategorisedText, Header, and ListItem category will not be used. ALl the other categories are useful for retreival.

* To create chunk we first need to combine all the elements in a meaningful way that provides semantic understanding of data.

* Once we combine all the content, we will be doing token aware chunking so that no sentence is broken or chunked that doesn't provide accurate information.

#### Before we generate embeddings we need to preprocess the extracted information and use text splitter for better chunking.

## Let's combine the content in semantically.

##### Just for reference 😜
page_content='Article' metadata={'source': '/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/Documents/cells-11-02650-v2.pdf', 'detection_class_prob': 0.8227533102035522, 'is_extracted': 'true', 'coordinates': {'points': ((np.float64(97.11119842529297), np.float64(290.358642578125)), (np.float64(97.11119842529297), np.float64(321.78399611111104)), (np.float64(178.90414428710938), np.float64(321.78399611111104)), (np.float64(178.90414428710938), np.float64(290.358642578125))), 'system': 'PixelSpace', 'layout_width': 1654, 'layout_height': 2339}, 'last_modified': '2025-11-01T08:41:48', 'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 1, 'file_directory': '/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/Documents', 'filename': 'cells-11-02650-v2.pdf', 'category': 'NarrativeText', 'element_id': 'be3a187cfe3ab16d142f7c7dfc25ceeb'}

The Output I want is :

Document(page_content = "semantic content of max chunk size", metadata = {"source":***, "page_number":***, })

In [ ]:
from hashlib import new
from langchain_core.documents import Document
import re

#def combine_semantic_content(docs: List[Document], max_chunk_size: int = 5000):
All_saemantic_docs = []
Accumulated_semantic_content = ""
accumulated_metadata = {}
section_header = "Pulsed Electrical Stimulation Affects Osteoblast Adhesion and Calcium Ion Signaling"
pattern = r'^(?:\d+(?:\.\d+)*\.\s+[A-Za-z ]+|[A-Za-z ]+)$'
necessaery_metadata = {'source', 'section_header', 'page_number', 'coordinates', 'category', 'image_base64', 'image_mime_type','text_as_html'}

# Iterating over documents
for doc in docs:
    metadata_copy = (doc.metadata).copy()
    doc_based_content = doc.page_content.strip()
    current_category = doc.metadata.get('category')

    is_header = current_category in ['Title'] # checking if it's a section header

    # If it a section header then we just add a new key into meatadata with header name
    if is_header:
        # If the content is already there then we add only new key in metadata
        if Accumulated_semantic_content:
            new_metadata = accumulated_metadata.copy()
            new_metadata['section_header'] =  section_header
            # creating new Document object
            new_docobj = Document(metadata=new_metadata, page_content= Accumulated_semantic_content)
            All_saemantic_docs.append(new_docobj)

        # if it's just strating of content, first we set the section header.(we are not writting content yet.)
        Refined_header = re.match(pattern, doc_based_content)
        if Refined_header:
            section_header = doc_based_content
        Accumulated_semantic_content=""
        accumulated_metadata = metadata_copy
        continue ## Move to the next Document Object
    if current_category in ['NarrativeText', 'Table', 'Image', 'FigureCaption']:

        header = ""
        cat=""
        if not Accumulated_semantic_content:
            header = f"\n\n# {section_header}\n {current_category}\n"
        semantic_content = f'{header} {cat} {doc_based_content}'
        #revised_metadata_copy = {key: value for key, value in metadata_copy.items() if key in necessaery_metadata}
        
        accumulated_metadata.update(metadata_copy)

        if len(Accumulated_semantic_content) + len(semantic_content) > 20000:
            new_metadata = accumulated_metadata.copy()
            new_metadata['section_header'] =  section_header
            # creating new Document object
            new_docobj = Document(metadata=new_metadata, page_content= Accumulated_semantic_content)
            All_saemantic_docs.append(new_docobj)

            # starting new chunk
            Accumulated_semantic_content = semantic_content

        else:
            Accumulated_semantic_content += semantic_content

# if the iteration is finished we put all the document objects in combined docs
if Accumulated_semantic_content:
    new_metadata = accumulated_metadata.copy()
    new_metadata['section_header'] = section_header
    new_docobj = Document(metadata=new_metadata, page_content= Accumulated_semantic_content)
    All_saemantic_docs.append(new_docobj)


for doc in All_saemantic_docs[2:5]:
    print(doc.page_content)
        




# Pulsed Electrical Stimulation Affects Osteoblast Adhesion and Calcium Ion Signaling
 Image
  f cells    Article


# G check for updates
 NarrativeText
  Citation: Staehlke, S.; Bielfeldt, M.; Zimmermann, J.; Gruening, M.; Barke, I.; Freitag, T.; Speller, S.; Van Rienen, U.; Nebe, B. Pulsed Electrical Stimulation Affects Osteoblast Adhesion and Calcium Ion Signaling. Cells 2022, 11, 2650. https://doi.org/ 10.3390/cells11172650  Academic Editor: Ohad Medalia  Abstract: An extensive research ﬁeld in regenerative medicine is electrical stimulation (ES) and its impact on tissue and cells. The mechanism of action of ES, particularly the role of electrical parameters like intensity, frequency, and duration of the electric ﬁeld, is not yet fully understood. Human MG-63 osteoblasts were electrically stimulated for 10 min with a commercially available multi-channel system (IonOptix). We generated alternating current (AC) electrical ﬁelds with a voltage of 1 or 5 V and frequencies of 7.9 or 2

#### Now I have whole semantic content. I was thinking to add image and table summary where the table and image is present.



## Creating a function to generate summary by taking table html content.

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
model = init_chat_model("openai:gpt-5.1")


/home/vraj/Desktop/CellSense/X20251016-CellSense-VB/CellSense/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.messages import SystemMessage, HumanMessage, AIMessage

Role = """You are an expert biomedical research assistant specialising in analysing scientific literature. 
You must generate concise and informative summaries of tables present in scientific documents. 
Your summaries should capture the key insights, trends, and significant data points from the table content provided to you. 
Ensure that your summaries are clear, accurate, and useful for researchers looking to understand the data quickly."""

table_template = ChatPromptTemplate([
    (SystemMessage, Role),
    (HumanMessage, "Generate a insightful summary for given table: \n{table_content}")
])

